# Copy Number Pipeline

In [492]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np
from collections import Counter

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from JKBio.helper.google_sheet import GSheet
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Loading BokehJS ...

## boot up

- you first need to go to [taiga](https://cds.team/taiga/dataset) and create some new datasets for the virtual release
- the easiest way to create a new dataset is to upload an empty file (since at least one file is required). This empty file can be deleted when you update the dataset with a new version

we are instanciating all the parameters needed for this pipeline to run

In [158]:
samplesetname = "20Q4"
prevname="20Q2"
prevversion=42
prevprevname ='20Q2'
prevprevversion= 40
virtual_public='public-20q3-3d35'
virtual_dmc='dmc-20q3-033d'
virtual_internal='internal-20q3-00d0'

refworkspace="broad-firecloud-ccle/DepMap_WES_CN_hg38"

genelist_hg38 = 'ftp://ftp.ncbi.nlm.nih.gov/pub/CCDS/current_human/CCDS.20180614.txt'

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"

release = samplesetname

In [3]:
%%R
# same thing for R
release <- '20Q4'
prevname <- '20Q3'
version<-37
prevversion <- 40
prevprevversion<- 38
prevprevname <- '20Q2'
genome_version <- 'hg38'

In [4]:
# we initialize the workspaces manager from dalmatian
refwm = dm.WorkspaceManager(refworkspace)

In [405]:
potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()

In [53]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame().set_index('cds_sample_id')

In [7]:
ccle_refsamples = pd.read_csv('temp/updated_ref_samples.csv', index_col=0)

## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the copy number dataset:

*   **BamToUnmappedRGBams_MC** vdauwera/BamToUnmappedRGBamsSnapshot ID: 3
*   **Generate_uBAM_File_List** gkugener/ArrayOfFilesToTxtSnapshot ID: 1
*   **Realign_WES_GATK4** gatk/PreProcessingForVariantDiscovery_GATK4Snapshot ID: 7
*   **CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9
*   **Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2

This output file for download will be saved under the sample set under the combined_seg_file attribute.

There are several other tasks in this workspace. In brief:

*   **CNV_Somatic_Panel_Workflow_Agilent_XX** gatk/CNV_Somatic_Panel_WorkflowSnapshot ID: 11. This task was used in this workspace to generate the Sanger PON. In the Sanger dataset, there is a set of 40 normal cell lines samples (cell lines derived from matched normal tissue). We can use these to generate a PON to normalize to rather than using the Agilent PON we use for the other CCLE cell lines. This leads to less noisy results. HOWEVER, results using the PON from this workflow should not use the X chromosome, as the sanger normals are not exclusively female or male (it is likely a mix).
*   **SANGER_PON_CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9. Same as the CNV_sample_XX_gatk, except that is uses the Sanger based PON. Should be used only for the Sanger cell lines.
*   **Sanger_PON_Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2. Aggregates the segment files for the samples that were run using the Sanger PON based CNV workflow.

## On Terra

In [8]:
# a list of Terra workflows that are in the workspace and that we will call sequentially
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [9]:
# see dalmatian
subid = refwm.create_submission(bamtoubam,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

Successfully created submission bc7f4de0-c19d-4794-859e-bce10dc6e90a.


RefreshError: ('<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 502 (Server Error)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-mo

In [10]:
subid = refwm.create_submission(ubamtofilelist,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

Successfully created submission fdccb21c-7689-444c-befe-c799513d31c0.
1.0 of jobs Succeeded in submission 0.sion 0. 12 mn elapsed.d.


[]

In [11]:
subid = refwm.create_submission(realign,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

Successfully created submission ce837346-291b-4482-ae83-449e5e1c53e1.
CDS-H4hPhD Failed for 0 jobs in submission 0. 3 mn elapsed.
0.9975845410628019 of jobs Succeeded in submission 0. elapsed.d.


RuntimeError: 1 failed submission

In [16]:
# Testing out the XY PoN for CN characterization. Will test by producing an output in a different column from usual so it's easy to delete the column attribute later
# Also, need to make a split between Agilent and ICE samples..
submission_id= refwm.create_submission("CNV_sample_XY_ice",etype='sample_set',entity=samplesetname, expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

Successfully created submission 343d14e3-b65a-437f-bdf3-75dd4437c712.
1.0 of jobs Succeeded in submission 0.sion 0. 107 mn elapsed.d.


[]

### copy pairs data to sample data

In [76]:
pairs = refwm.get_pairs()

In [79]:
pairs = pairs[~pairs['called_copy_ratio_segments_tumor'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [81]:
refwm.update_sample_attributes(pairs)

Successfully updated attributes '['read_counts_tumor', 'scaled_delta_MAD_normal', 'copy_ratio_parameters_begin_normal', 'denoised_copy_ratios_tumor', 'normal_het_allelic_counts_tumor', 'scaled_delta_MAD_tumor', 'denoised_MAD_normal', 'denoised_copy_ratios_plot_tumor', 'denoised_MAD_tumor', 'allele_fraction_parameters_begin_normal', 'modeled_segments_begin_tumor', 'het_allelic_counts_tumor', 'modeled_segments_begin_normal', 'standardized_MAD_normal', 'copy_ratio_only_segments_tumor', 'read_counts_entity_id_tumor', 'called_copy_ratio_segments_tumor', 'allele_fraction_legacy_segments_tumor', 'modeled_segments_tumor', 'allele_fraction_legacy_segments_normal', 'copy_ratio_parameters_tumor', 'allele_fraction_parameters_begin_tumor', 'oncotated_called_file_tumor', 'copy_ratio_only_segments_normal', 'modeled_segments_normal', 'allele_fraction_parameters_normal', 'modeled_segments_plot_tumor', 'standardized_copy_ratios_tumor', 'allelic_counts_entity_id_tumor', 'denoised_copy_ratios_normal', 'co

continuing

In [85]:
submission_id = refwm.create_submission("Aggregate_CN_seg_files",entity="all")
terra.waitForSubmission(refworkspace,submission_id)

Successfully created submission 7be075b0-caa2-4458-8e7d-5ac598b3ae94.
1.0 of jobs Succeeded in submission 0.sion 0. 62 mn elapsed.


[]

__we are getting the results file path__

In [87]:
terra.waitForSubmission(refworkspace,submission_id)
aggregated = refwm.get_entities('sample_set').loc['all']["combined_seg_file"]
aggregated

1.0 of jobs Succeeded in submission 0.


'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/7be075b0-caa2-4458-8e7d-5ac598b3ae94/aggregate_CN_segments_wrkflw/816c5c6f-e61b-4a94-8fd9-e246df948ef2/call-aggregate_CN_segments/all.called.seg'

## On local

__We then save the workflow configurations used__

In [21]:
terra.saveConfigs(refworkspace,'data/'+samplesetname+'/CNVconfig')

__delete unmapped bams generated during the process__

In [32]:
toremove = ["readgroup_ubams",]
res = refwm.get_samples()
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

Successfully deleted attribute readgroup_ubams for 2334 samples.


In [ ]:
# sometimes the previous step does not work and you need to do it manually (you can run this to check it worked)
for val in samplesinset.readgroup_ubams:
    ubams = ''
    if not type(val) is list:
        continue 
    for v in val:
        ubams+=' '+v
    os.system('gsutil -m rm'+ubams)

__and move the hg38 aligned bams to our own datastorage bucket__

Note that we may encounter some WGS files, which need to go to a different folder from the WES bam files.

In [48]:
onlycol = ['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index', 'hg38_analysis_ready_bam_md5']
samplesinset= [i['entityName'] for i in refwm.get_entities('sample_set').loc[samplesetname].samples]
wes_newgs = 'gs://cclebams/hg38_wes/'
wes_res, flagged = terra.changeGSlocation(refworkspace, newgs=wes_newgs, onlysamples=samplesinset, onlycol=onlycol, entity='sample', keeppath=False, dry_run = False)

using the data from broad-firecloud-ccle/DepMap_WES_CN_hg38 sample list
this should only contains gs:// paths otherwise precise columns using "onlycol"
Successfully updated attributes '['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index', 'hg38_analysis_ready_bam_md5']' for 414 samples.


#### set it this way in our sample tracker

In [ ]:
ccle_refsamples = pd.read_csv('temp/updated_ref_samples.csv', index_col=0)

In [431]:
ccle_refsamples.loc[wes_res.index.tolist()][['legacy_bam_filepath','legacy_bai_filepath']]

you need to have installed JKBio in the same folder as ccle_processing


,legacy_bam_filepath,legacy_bai_filepath
cds_sample_id,,
CDS-0nxrUZ,NaN,NaN
CDS-0o3uUk,NaN,NaN
CDS-0qPmaJ,NaN,NaN
CDS-13Wsxa,NaN,NaN
CDS-1i4BK0,NaN,NaN
...,...,...
CDS-ziEOXJ,NaN,NaN
CDS-ZMsoXe,NaN,NaN
CDS-ZN0K5L,NaN,NaN


In [434]:
ccle_refsamples.loc[wes_res.index.tolist(),['legacy_bam_filepath','legacy_bai_filepath']] = ccle_refsamples.loc[wes_res.index.tolist()][['internal_bam_filepath','internal_bai_filepath']]
ccle_refsamples.loc[wes_res.index.tolist(),['internal_bam_filepath','internal_bai_filepath']] = wes_res[['hg38_analysis_ready_bam','hg38_analysis_ready_bam_index']]
ccle_refsamples.loc[wes_res.index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(wes_res['hg38_analysis_ready_bam'].tolist(),'-l')]
ccle_refsamples.loc[wes_res.index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(wes_res['hg38_analysis_ready_bam'].tolist(),'-L')]
ccle_refsamples.loc[wes_res.index.tolist(),'md5_hash'] = gcp.catFiles(wes_res['hg38_analysis_ready_bam_md5'].tolist(), cut=32)

0.0
0.12077294685990339
0.24154589371980678
0.3623188405797102
0.48309178743961356
0.6038647342995169
0.7246376811594204
0.8454106280193238
0.9661835748792271


### Get QC files

In [ ]:
toprocess = refwm.get_samples().index.tolist() #wes_res.index.tolist()

In [91]:
dataBam = getQC(workspace=refworkspace ,only=[], qcname=["hg38_duplication_metrics","hg38_bqsr_report"])
dataCN = getQC(workspace=refworkspace ,only=[], qcname=["allelic_counts_tumor","delta_MAD_tumor","denoised_MAD_tumor","scaled_delta_MAD_tumor","denoised_copy_ratios_lim_4_plot_tumor","denoised_copy_ratios_plot_tumor","modeled_segments_plot_tumor"])

In [93]:
for k,v in dataCN.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'processing_qc'] = str(v)
for k,v in dataBam.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'bam_qc'] = str(v)

__We download and reprocess removing the appended version and keeping only the newest versions__

In [88]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

Copying gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/7be075b0-caa2-4458-8e7d-5ac598b3ae94/aggregate_CN_segments_wrkflw/816c5c6f-e61b-4a94-8fd9-e246df948ef2/call-aggregate_CN_segments/all.called.seg...
/ [1 files][ 45.8 MiB/ 45.8 MiB]                                                
Operation completed over 1 objects/45.8 MiB.                                     


In [397]:
segments = pd.read_csv("temp/cnv_ccle.called.seg", sep='\t')
# TODO: copy allelic calls as well

In [90]:
len(set(segments.Sample))

2332

In [398]:
segments = segments.rename(columns={'CONTIG':'Chromosome',
'START':'Start',
'END':'End',
'Sample':"DepMap_ID",
'NUM_POINTS_COPY_RATIO':'Num_Probes',
'MEAN_LOG2_COPY_RATIO':'Segment_Mean',
'CALL':'Status'})
segments['Source'] = "Broad WES"

### Priorization

add columns to seg file with arxspan ID, version. only keep the newest version for any given arxspan ID.
The process to keep the newest version of any given line is a little different from 20Q2 onwards, because don't have any dataset that uses the CDS-IDs for the data from 20Q1 or earlier.

We have to download the Taiga datasets from the previous quarter, see if we have any arxspan IDs with new data, and then replace with that data. We use the function called "removeOlderVersions" to do this.

In [121]:
renaming = removeOlderVersions(names=set(segments.DepMap_ID.tolist()), refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")

removed 627 duplicate samples


## post Procesing

The post processing happens in R using guillaume's functions, in brief:

- processSegments
- filterForCCLE
- interpolateGapsInSegmented
- extendEndsOfSegments
- reprioritizeData

In [221]:
#legacy_segments = tc.get(name='depmap-wes-cn-data--08f3', file='legacy_segments')
legacy_segments = pd.read_csv('data/legacy_segments.csv')

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [373]:
prevcounts= tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.20Q2.gene')

In [410]:
notWES = set(potential_list) - set(renaming.values())
notWES

In [417]:
notWESnotlegacy = notWES - set(legacy_segments.DepMap_ID)
%store notWESnotlegacy
notWESnotlegacy

Stored 'notWESnotlegacy' (set)


{'ACH-001714',
 'ACH-002014',
 'ACH-002512',
 'ACH-002709',
 'ACH-002874',
 'ACH-002875'}

In [210]:
prevsegment = prevsegment[~prevsegment.DepMap_ID.isin(["ACH-001189", "ACH-002303", "ACH-002315", "ACH-002335", "ACH-002341"])]

In [116]:
cyto = pd.read_csv('data/hg38_cytoband.gz', sep='\t',names=['Chromosome', 'Start', 'End','loc','stains'])
genelist = pd.read_csv(genelist_hg38,sep='\t')
hgnc_complete_set = tc.get(name='hgnc-87ab', file='hgnc_complete_set')
gene_mapping = pd.read_csv('data/genemapping_19Q1.csv')

In [197]:
gene_mapping = tc.get(name='depmap-wes-cn-data--08f3', file='WES_CN_hg38_gene_DM19Q1')
gene_mapping = gene_mapping[gene_mapping.columns[:5]]
gene_mapping = gene_mapping.rename(columns={'EGID':'ensembl_id','SYMBOL':'symbol','CHR':'Chromosome','CHRLOC':'start','CHRLOCEND':'end'})

In [217]:
gene_mapping.to_csv('data/genemapping_19Q1.csv', index=None)

In [399]:
segments['Chromosome'] = [i[3:] for i in segments['Chromosome'].tolist()]
segments = segments.sort_values(by=['DepMap_ID','Chromosome','Start','End'])
gene_mapping = gene_mapping.sort_values(by=['Chromosome','start','end'])

In [401]:
segments.to_csv('temp/WES_segments_allreplicates_'+samplesetname+'.csv', index=None)

In [423]:
gapmergedsegs = manageGapsInSegments(segments)

you need to have installed JKBio in the same folder as ccle_processing


In [424]:
segments

,DepMap_ID,Chromosome,Start,End,Num_Probes,Segment_Mean,Status,Source
0,CDS-00rz9N,1,925689,55069357,6119,-0.060818,0,Broad WES
1,CDS-00rz9N,1,55071563,55215339,40,0.715102,+,Broad WES
2,CDS-00rz9N,1,56496298,121177568,3084,-0.092687,0,Broad WES
3,CDS-00rz9N,1,143880473,145736386,33,1.106529,+,Broad WES
4,CDS-00rz9N,1,145738233,154945344,1168,0.558489,+,Broad WES
...,...,...,...,...,...,...,...,...
969647,CDS-zzbiLf,Y,20584221,20584776,1,-29.851065,-,Broad WES
969648,CDS-zzbiLf,Y,20589231,20756418,2,-5.208849,-,Broad WES
969649,CDS-zzbiLf,Y,20756525,20761633,3,-22.304713,-,Broad WES
969650,CDS-zzbiLf,Y,20768552,20779918,2,-6.268679,-,Broad WES


In [426]:
genecn = toGeneMatrix(gapmergedsegs, gene_mapping)

In [428]:
genecn = np.log2((2**genecn)+1)
genecn.mean(1)

In [440]:
a = set(prevcounts.columns) & set(genecn.columns)

In [442]:
simi = np.corrcoef(genecn[a],prevcounts[a])

In [452]:
simi = genecn[a].T*prevcounts[a]

In [453]:
simi.shape

(28891, 29456)

In [470]:
set(genecn.index) - set(ccle_refsamples[ccle_refsamples.datatype=='wes'].index)

set()

In [515]:
genecn.max(1).max()

12.552972098551775

In [516]:
2**12.5

5792.618751480198

In [514]:
prevcounts.max(1).max()

7.220474335775503

In [517]:
2**7

128

In [469]:
genecn = genecn.drop(index={'CDS-8Ut3sT',
 'CDS-Ip02tY',
 'CDS-Rd4nMx',
 'CDS-TGTiB8',
 'CDS-VnMBYD',
 'CDS-YSRYLi',
 'CDS-ZJh6UN',
 'CDS-dgxjAa'})

In [458]:
genecn.shape

(2332, 27562)

In [479]:
corr = {i:{} for i in prevcounts.index.tolist()}
ge = genecn[a]
pre =prevcounts[a]
for k,val in ccle_refsamples[ccle_refsamples.datatype=='wes'][['arxspan_id','version']].iterrows():
    if val[0] in pre.index and k in ge.index:
        corr[val[0]].update({val[1]:pearsonr(ge.loc[k],pre.loc[val[0]])[0]})

In [507]:
toreplace = []
touse = []
tomerge = []
perfect= []
for k, val in corr.items():
    if len(val)==0:
        touse.append((k,1))
        continue
    m = max(val.values())
    if m<0.85:
        toreplace.append(k)
    else:
        for i ,e in val.items():
            if e==m:
                touse.append((k,i))
                if e>0.99:
                    perfect.append((k,i))
    if len(val)>2:
        if max(Counter(list(val.values())).values())>1:
            tomerge.append(k)

In [ ]:
remove X and Y
try reprocessing (without Y) o guillaume#'s pipeline

In [502]:
corr

{'ACH-000001': {1: 0.8862356155398592},
 'ACH-000002': {1: 0.7453020350453161},
 'ACH-000003': {},
 'ACH-000004': {2: 0.8780160121269522,
  1: 0.7912059833932297,
  3: 0.8780160121269522},
 'ACH-000005': {1: 0.9565852263529243, 2: 0.9565852263529243},
 'ACH-000006': {1: 0.8406546252491695},
 'ACH-000007': {2: 0.9132008556769335,
  1: 0.8364482167420976,
  3: 0.9132008556769335},
 'ACH-000008': {1: 0.7624421021085812},
 'ACH-000009': {1: 0.872116813824773},
 'ACH-000011': {1: 0.7952921135564988},
 'ACH-000012': {1: 0.8676301253955646},
 'ACH-000013': {1: 0.9189515830871217},
 'ACH-000014': {},
 'ACH-000015': {1: 0.8568946306390246},
 'ACH-000016': {},
 'ACH-000017': {1: 0.933130017813276, 2: 0.933130017813276},
 'ACH-000018': {1: 0.7127556164716891},
 'ACH-000019': {2: 0.9430264142175129,
  1: 0.7395313345941336,
  3: 0.9430264142175129},
 'ACH-000020': {2: 0.8222848798593649, 1: 0.7671499845298603},
 'ACH-000021': {1: 0.9259588645900585},
 'ACH-000022': {1: 0.8767552791274907, 2: 0.876

In [505]:
len(toreplace)

842

In [506]:
len(touse)

1160

In [504]:
tomerge

['ACH-000004',
 'ACH-000007',
 'ACH-000019',
 'ACH-000047',
 'ACH-000200',
 'ACH-000217',
 'ACH-000247',
 'ACH-000304',
 'ACH-000312',
 'ACH-000337',
 'ACH-000434',
 'ACH-000452',
 'ACH-000466',
 'ACH-000514',
 'ACH-000544',
 'ACH-000557',
 'ACH-000577',
 'ACH-000596',
 'ACH-000655',
 'ACH-000672',
 'ACH-000698',
 'ACH-000740',
 'ACH-000767',
 'ACH-000800',
 'ACH-000823',
 'ACH-000837',
 'ACH-000866',
 'ACH-001075',
 'ACH-001129',
 'ACH-001151',
 'ACH-001190',
 'ACH-001333',
 'ACH-001336',
 'ACH-001341',
 'ACH-001345',
 'ACH-001360',
 'ACH-001368',
 'ACH-001373',
 'ACH-001374',
 'ACH-001401',
 'ACH-001402',
 'ACH-001418',
 'ACH-001443',
 'ACH-001496',
 'ACH-001497',
 'ACH-001500',
 'ACH-001517',
 'ACH-001525',
 'ACH-001526',
 'ACH-001530',
 'ACH-001542',
 'ACH-001549',
 'ACH-001630',
 'ACH-001638',
 'ACH-001642',
 'ACH-001650',
 'ACH-001654',
 'ACH-001655',
 'ACH-001674',
 'ACH-001702',
 'ACH-001715',
 'ACH-001794',
 'ACH-001861',
 'ACH-002446',
 'ACH-001398',
 'ACH-001453',
 'ACH-0014

merge:
ACH-000005 v1 and v2
ACH-000017
ACH-000022
ACH-000029
ACH-000005

In [488]:
count

969652

1

In [480]:
corr

{'ACH-000001': {1: 0.8862356155398592},
 'ACH-000002': {1: 0.7453020350453161},
 'ACH-000003': {},
 'ACH-000004': {2: 0.8780160121269522,
  1: 0.7912059833932297,
  3: 0.8780160121269522},
 'ACH-000005': {1: 0.9565852263529243, 2: 0.9565852263529243},
 'ACH-000006': {1: 0.8406546252491695},
 'ACH-000007': {2: 0.9132008556769335,
  1: 0.8364482167420976,
  3: 0.9132008556769335},
 'ACH-000008': {1: 0.7624421021085812},
 'ACH-000009': {1: 0.872116813824773},
 'ACH-000011': {1: 0.7952921135564988},
 'ACH-000012': {1: 0.8676301253955646},
 'ACH-000013': {1: 0.9189515830871217},
 'ACH-000014': {},
 'ACH-000015': {1: 0.8568946306390246},
 'ACH-000016': {},
 'ACH-000017': {1: 0.933130017813276, 2: 0.933130017813276},
 'ACH-000018': {1: 0.7127556164716891},
 'ACH-000019': {2: 0.9430264142175129,
  1: 0.7395313345941336,
  3: 0.9430264142175129},
 'ACH-000020': {2: 0.8222848798593649, 1: 0.7671499845298603},
 'ACH-000021': {1: 0.9259588645900585},
 'ACH-000022': {1: 0.8767552791274907, 2: 0.876

In [430]:
set(prevcounts.index) - set(genecn.index)

{'ACH-000176',
 'ACH-000437',
 'ACH-000481',
 'ACH-001225',
 'ACH-001687',
 'ACH-000340',
 'ACH-000549',
 'ACH-000134',
 'ACH-001561',
 'ACH-000616',
 'ACH-002100',
 'ACH-000465',
 'ACH-001848',
 'ACH-000250',
 'ACH-000956',
 'ACH-002025',
 'ACH-002146',
 'ACH-001228',
 'ACH-002024',
 'ACH-001973',
 'ACH-001856',
 'ACH-000937',
 'ACH-000387',
 'ACH-000987',
 'ACH-001959',
 'ACH-002046',
 'ACH-000427',
 'ACH-000830',
 'ACH-000776',
 'ACH-000600',
 'ACH-000275',
 'ACH-001388',
 'ACH-000317',
 'ACH-001839',
 'ACH-000624',
 'ACH-000706',
 'ACH-000982',
 'ACH-000306',
 'ACH-000456',
 'ACH-000686',
 'ACH-002135',
 'ACH-002169',
 'ACH-000205',
 'ACH-002228',
 'ACH-001496',
 'ACH-000523',
 'ACH-002293',
 'ACH-000213',
 'ACH-000222',
 'ACH-000538',
 'ACH-000477',
 'ACH-000735',
 'ACH-000601',
 'ACH-001850',
 'ACH-002051',
 'ACH-000615',
 'ACH-002310',
 'ACH-002101',
 'ACH-002093',
 'ACH-000417',
 'ACH-002397',
 'ACH-000475',
 'ACH-000564',
 'ACH-000859',
 'ACH-002090',
 'ACH-001200',
 'ACH-0003

#### saving samples used for 20Q2

In [ ]:
a_indexed[a_indexed.index.isin(renaming.keys())].rename(renaming).to_csv("temp/cnv_ccle.called.seg", sep='\t')

In [ ]:
ccle_refsamples.loc[renaming.keys(),version]=1
ccle_refsamples.to_csv('temp/updated_ref_samples.csv',index_col=0)

In [107]:
segments = segments.set_index('Sample')
segments.index.names = ['DepMap_ID']
segments

,CONTIG,START,END,NUM_POINTS_COPY_RATIO,MEAN_LOG2_COPY_RATIO,CALL
DepMap_ID,,,,,,
CDS-00rz9N,chr1,925689,55069357,6119,-0.060818,0
CDS-00rz9N,chr1,55071563,55215339,40,0.715102,+
CDS-00rz9N,chr1,56496298,121177568,3084,-0.092687,0
CDS-00rz9N,chr1,143880473,145736386,33,1.106529,+
CDS-00rz9N,chr1,145738233,154945344,1168,0.558489,+
...,...,...,...,...,...,...
CDS-zzbiLf,chrY,20584221,20584776,1,-29.851065,-
CDS-zzbiLf,chrY,20589231,20756418,2,-5.208849,-
CDS-zzbiLf,chrY,20756525,20761633,3,-22.304713,-


## Validation step

Once the files are saved, we load them back in python and do some validations, in brief:

- mean,max,var...
- to previous version: same mean,max,var...
- checkAmountOfSegments: flag any samples with a very high number of segments
- checkGeneChangeAccrossAll: flag any genes which stay at a similar value across all samples

In [ ]:
segmentcn = pd.read_csv('temp/wes.'+release+'.segment.cn', sep = ',')

In [ ]:
# getting the previous versions to check that we have everything we should
prev = set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.'+prevname+'.gene', version=prevversion).index.tolist())
prevprev= set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.'+prevprevname+'.gene', version=prevprevversion).index.tolist())

In [ ]:
new1 = set(genecn.index.values.tolist())
new2 = set(segmentcn['DepMap_ID'].values.tolist())
print(len(prev), len(prev & new1), len(new1), len(new1 & new2))

In [ ]:
segmentcn[segmentcn.DepMap_ID=="ACH-001546"]

In [ ]:
"ACH-001546" in segmentcn.index.tolist()

In [ ]:
checkAmountOfSegments(segmentcn,thresh = 750)

In [ ]:
checkGeneChangeAccrossAll(genecn, thresh=1.5)

In [ ]:
segmentcn.Start = segmentcn.Start.astype(int)
segmentcn.End = segmentcn.End.astype(int)

In [ ]:
genecn.values.min(), genecn.values.mean(), genecn.values.max()

In [ ]:
if(genecn.values.max() > 100):
    print("\n\n\nTOO HIGH, not LOG2 transformed!")
if(len(genecn.index.tolist()) > len(set(genecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
for k, val in samplesinset.iterrows():
    plot = val["modeled_segments_plot_tumor"]
    ! gsutil cp $plot temp/
    print(k)
    print(val['arxspan_id'])
    display(Image('temp/'+plot.split('/')[-1]))

These look bad in 20Q1: 
ACH-002511 (M140325), ACH-001370 (OCIP5X)

These CN plots subjectively appear to have too many segments in new 20Q2 samples: 
ACH-002399 (CDS-sukIAT, 21NT_1), ACH-002401 (CDS-tVy3GF, 21MT2_1), ACH-002400 (CDS-VUHMHG, 21MT1_1)

In [ ]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_embargo = [i for i in gsheets['WES_embargo'].values.tolist() if str(i) != 'nan']
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if str(i) != 'nan']
blacklist = [i for i in gsheets['blacklist'].values.tolist() if str(i) != 'nan']

wes_embargo, wes_dmc_embargo, blacklist

# Upload to taiga

- we load the blacklisted/embargoed sample ids
- we log2 transform and create a file for each release (and one containing everything)
- we upload the files using taigapy in a corresponding taiga dataset with the corresponding description and also upload it to its virtual dataset

## we push full dataset version in depmap taiga CN

In [ ]:
print(segmentcn.Segment_Mean.max(), segmentcn.Segment_Mean.mean(), segmentcn.Segment_Mean.min())
print(genecn.values.max(), genecn.values.mean(),genecn.values.min())

In [ ]:
## for genecn removing first blacklisted, then embargoed, to create two datasets
genecn = genecn.apply(lambda x: np.log2(1+x))
genecn.to_csv('temp/wes.'+release+'.gene.cn', index=True)
segmentcn.to_csv('temp/wes.'+release+'.segment.cn', index=False)

In [ ]:
if(genecn.values.max() > 100):
    print("\n\n\nTOO HIGH, not LOG2 transformed!")
if(len(genecn.index.tolist()) > len(set(genecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
print("Old genecn shape:", genecn.shape)
print("Old segmentcn shape:", segmentcn.shape)

In [ ]:
tc.update_dataset(dataset_permaname="segmented-cn-wes-prioritzed-7fe1", 
                  upload_file_path_dict={
                    'temp/wes.'+release+'.gene.cn': 'NumericMatrixCSV',
                    'temp/wes.'+release+'.segment.cn': 'TableCSV'},
                  changes_description=
"""
Removing duplication of ACH-000219 from segmentcn file to prevent issues in future releases, which use this Taiga dataset in the process of determining which new lines to release. Now both the genecn file and the segmentcn file have 1767 unique DepMap IDs.
""",
                  dataset_description=
"""
# Copy Number

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, log2(x+1) transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

These data are generated for Achilles to pull from to run CERES.

Versions to use:

v45 for 20Q2 (For Achilles QC, use v44 of segmentcn)
v40 for 20Q1
v38 for 19Q4
v33 for 19Q4
v25 for 19Q2 (hg38 aligned, Broad WES and Sanger WES based calls were generated from bam realignment. SNP based calls are still from liftover). The gene mapping script was updated to improve the gene level matrix (to remove NAs). The segment level matrix is untransformed relative CN. Gene level matrix is log2(CN + 1).
v20 for 19Q1 (version 21 is hg19)
v18: for 18Q4
v15: for 18Q3
v11: for 18Q2
Gene-level matrix in versions below 10 were using hg38 and not hg19. Version 11 is corrected and should be used instead

Calls on X, Y chromosome for profiles should not be used.

Prioritization is as follows:

Broad WES kept over everything
Sanger WES kept if:
This cell line did not fail fingerprinting
This cell line has no other CN data
This cell lines does not have CRISPR LFC data from the Achilles screen
This CN profile correlates better with Achilles CRISPR LFC data than Broad SNP CN OR the % gene-level difference between this cell lines CN profile from Sanger WES and Broad SNP < 2.5%
Broad SNP used for remaining lines with no Broad WES or with Sanger WES that does not pass the criteria above
The 'Source' column indicates which CN profile was used for that cell line.

version 6: renamed Sample column to CCLE_name for consistency for the Achilles pipeline

version 7: missing chordoma lines

version 8: fixed to names of two chordoma lines (changed suffix from CHORDOMA -> BONE) and removed renamed 
cell lines that were duplicated (with different names). Reran comparison using 18q2 LFC results. Gene level matrix will be generated for version 9

version 11: corrected error in gene-level matrix calculation (previously had been aligned to hg38 however alignment should be hg19). Segment level calls are unaffected.

versions 12-14: Sanger WES were multiplied by 2 so should not be used

version 15: internal segments and gene level matrices for 18q3 release including public version (removed black list lines and Broad WES < 6 months old). Gene level matrices are indexed using Broad IDs.

version 16: internal segments and gene level matrices for 18q4 release including public version (uses all SNP and only WES if those lines are present in the 18Q4 public Achilles dataset)

version 17: same as version 16 but with two additional line in the internal version

version 18: same as version 17 but switched one line in public to use SNP instead of WES because not in public Avana

version 20: two major changes occurred (1) we are using a FireCloud based pipeline for CN calling now for Broad WES data (2) we have moved to use hg38. This is accomplished by lifting over coordinates from hg19 to hg38 after processed by the CN pipeline.
v21 same as version 20, but we are using the original hg19 coordinates, not hg38

version 25: Broad WES and Sanger WES were realigned to hg38. SNP still uses liftover from hg19 to hg38

version 35: Seeing what went wrong with the upload.

version 36: problem with not log2 transforming the data

version 37: resolving the problem with log2 transforming the segment data

version 38: resolving the problem with log2 transforming the segment data

version 39: 20Q1. Samples ACH-002511 (M140325) and ACH-001370 (OCIP5X) appear to have too many segments looking at the CN profile.

version 40: unlog2 transforming segmentcn

version 41: 20Q2 (segmentcn is just relative copy number, whereas the genecn is log2(x+1) transformed). Added 7 new samples.
These CN plots subjectively appear to have too many segments in new 20Q2 samples: ACH-002399 (CDS-sukIAT, 21NT\_1), ACH-002401 (CDS-tVy3GF, 21MT2\_1), ACH-002400 (CDS-VUHMHG, 21MT1\_1)

version 42: **note: version 42 is missing some of the cell lines. Do not use** 

version 43: Resolving issue of no DepMap ID index in the genecn file. Duplicating the CN data in genecn and segmentcn for ACH-000219 so we have CN data for ACH-002874, the same cell line grown in different media. This step is required for Achilles / CERES.

version 44: Removing duplication of ACH-000219 from genecn file. The Achilles QC only needs the duplication in the segmentcn file. This change results in 1767 unique DepMap IDs in the genecn file, and 1768 unique DepMap IDs in the segmentcn file.

version 45: Removing duplication of ACH-000219 from segmentcn file to prevent issues in future releases, which use this Taiga dataset in the process of determining which lines should be released to Public. Now both the genecn file and the segmentcn file have 1767 unique DepMap IDs.

version 46: removing two weird undefined lines 

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean""")

## We push internal dataset with blacklisted removed

and we add it to eternal dataset and to virtual dataset

In [ ]:
## for segment removing first blacklisted, then embargoed, to create two datasets
print(len(segmentcn))
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
print(len(segmentcn))
segmentcn.to_csv('temp/internal_'+release+'_segs_cn', index=False)
print(len(genecn))
genecn = genecn[~genecn.index.isin(blacklist)]
print(len(genecn))
genecn.to_csv('temp/internal_'+release+'_gene_cn', index=True)

In [ ]:
prevcn = tc.get(name='depmap-cn-data-81a7', version=24, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.DepMap_ID))
print("new lines")
newlines = set(genecn.DepMap_ID) - set(prevcn.DepMap_ID) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname="depmap-wes-cn-data-81a7", 
                  upload_file_path_dict={
                    'temp/internal_'+release+'_gene_cn': 'NumericMatrixCSV',
                    'temp/internal_'+release+'_segs_cn': 'TableCSV'},
                  dataset_description=
"""
# Copy Number


## ** Version 1 Internal 18Q1****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', 
                                  data.version=5, 
                                  data.file='gene_CN_WES_priority')
source_info <- data.frame(ccle_name=gsub("snp_|sangerWES_|ccleWES_|achillesWES_", 
                                         "", row.names(wes_pri)), 
                          source=gsub("_.*", "", row.names(wes_pri)))
wes_pri %<>% magrittr::set_rownames(source_info$ccle_name)

```

## ** Version 2 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=9, data.file='wes_priority_cn_gene_matrix') %>% log2()


```

## ** Version 3 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
## ** Version 4-6 Internal 18Q3****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Version 5 has updated cell line name mapping

Version 4 and 5 the segment level CN for Sanger's data is off by a factor of 2, version 6 corrects this

**** Version 7 Internal 18Q4****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

## ** Version 8-9 Internal 19Q1****

version 9 has the correct data for 19Q1

## ** Version 10-11 Internal 19Q2****

__version 11 added an additional 13 cell lines and adds the segment level copy number data__

## ** Version 12 Internal 19Q3****

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__. 


## ** Version 15 Internal 19Q4****

Adding 35 new cell lines

## ** Version 16 Internal 19Q4****
resolving problem with not having log2 transform 

## ** Version 17 Internal 19Q4****
resolving problem with having log2 transform on segments

## ** Version 18 Internal 20Q1****
adding 8 new cell lines

## ** Version 19 Internal 20Q1****
unlog2 transforming segmentcn

## ** Version 20 Internal 20Q1****
adding new cell lines

## ** Version 21 Internal 20Q1****
reparing some missing lines

## ** Version 22 Internal 20Q2****
adding new lines

## ** Version 23 Internal 20Q3****
nothing changed from 20Q2 updating the blacklist

## ** Version 24 Internal 20Q3****
updating the blacklists


Some cells lines have been flagged as:

 - having bad looking copy ration plots = ACH-002511 (M140325) and ACH-001370 (OCIP5X)
 - having too many segments (format: sample seg_count) = ACH-001079 2586, ACH-000044 1202, ACH-000258 872, ACH-001230 947, ACH-000068 812, ACH-000454 1051, ACH-000216 925, ACH-001150 782, ACH-001214 889, ACH-002335 1312, ACH-000836 1001, ACH-001957 1426, ACH-000960 913, ACH-000458 762, ACH-000578 869, ACH-000327 819, ACH-000090 1024, ACH-000488 954, ACH-000848 1171, ACH-000923 1469, ACH-000904 868, ACH-000452 816, ACH-000600 939, ACH-001656 902, ACH-000854 899, ACH-000774 953, ACH-001000 980, ACH-000941 813, ACH-000887 1408, ACH-001017 1223, ACH-001171 792, ACH-001071 1175, ACH-000593 764, ACH-001239 851, ACH-000071 1287, ACH-001956 1368, ACH-000509 873, ACH-002204 1318, ACH-000550 974, ACH-000738 1064, ACH-000870 1557, ACH-001036 858, ACH-001043 825, ACH-000028 868, ACH-001955 1296, ACH-000419 826, ACH-001234 819, ACH-001094 1036, ACH-001225 792, ACH-000118 794, ACH-000300 1431, ACH-001113 1072, ACH-001045 822, ACH-000444 974, ACH-000901 816, ACH-000865 1358, ACH-000961 763, ACH-001249 1756, ACH-000167 838, ACH-001101 1005, ACH-000842 929, ACH-000837 1015, ACH-000710 968, ACH-000195 2029, ACH-000064 1203, ACH-000690 771, ACH-000635 1368, ACH-000356 1294, ACH-000659 1129, ACH-000868 1422, ACH-000128 767, ACH-000658 927, ACH-001088 1337
 - Genes having a similar CN value accross all: []
 
## ** Version 20 Internal 20Q2****
Added 7 samples.

Some cells lines have been flagged as:

 - having bad looking copy ratio plots (appear to have too many segments): ACH-002399 (CDS-sukIAT, 21NT\_1), ACH-002401 (CDS-tVy3GF, 21MT2\_1), ACH-002400 (CDS-VUHMHG, 21MT1\_1)
 - having too many segments (format: sample seg_count): same as for 20Q1
 - Genes having a similar CN value accross all samples: []
 
 
## ** Version 21 Internal 20Q2****
 
Duplicating the CN data in genecn and segmentcn for ACH-000219 so we have CN data for ACH-002874, the same cell line grown in different media. This step is required for Achilles / CERES.

Version 22: removing two weird undefined lines 

## ** Version 23 Internal 20Q3****

same  as  20Q2 for this release. no new lines

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean

NEW LINES:
"""+newlines)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])

## We add to dmc as in internal*

* **NOTE: change as of 20Q2 onwards**. We need to remove lines in WES_DMC_embargo from the Internal version of the CN datasets before we upload the `genecn` and `segmentcn` files to DMC.

In [ ]:
## for segment removing first blacklisted, then embargoed, to create two datasets
print(len(segmentcn))
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(wes_dmc_embargo)]
print(len(segmentcn))
segmentcn.to_csv('temp/dmc_'+release+'_segs_cn', index=False)
print(len(genecn))
genecn = genecn[~genecn.index.isin(wes_dmc_embargo)]
print(len(genecn))
genecn.to_csv('temp/dmc_'+release+'_gene_cn', index=True)

In [ ]:
prevcn = tc.get(name='depmap-cn-data-9b9d', version=13, file='dmc_'+prevname+'_segs_cn')
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.index))
print("new lines")
newlines = set(genecn.index) - set(prevcn.DepMap_ID) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname="depmap-cn-data-9b9d",
                upload_file_path_dict={
                    'temp/dmc_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/dmc_'+release+'_segs_cn': 'TableCSV',
                   },
                  changes_description=
"""
Adding 20Q2 samples.
""",
                
                  dataset_description="""
**** Version 1-2 DMC 19Q1****

version 2 contains the correct data for 19Q1

**** Version 3-4 DMC 19Q2****

__version 4 added an additional 13 cell lines and adds the segment level copy number data__

**** Version 5 DMC 19Q3***

**** Version 7 DMC 19Q4***
adding 35 new cell lines

**** Version 8 DMC 19Q4****
resolving problem with not having log2 transform 

**** Version 9 DMC 19Q4****
resolving problem with having log2 transformed the segments

**** Version 10 DMC 20Q1****
adding new samples

**** Version 11 DMC 20Q1****
unlog2 transforming segmentcn

**** Version 12 DMC 20Q2****
Adding samples to be included in 20Q2

**** Version 13 DMC 20Q2****
unknown changes

**** Version 14 DMC 20Q3****
updated blacklists

**** Version 15 DMC 20Q3****
issues with blacklists

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__.  The segment copy number data includes the mean segment copy number segments.

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean

NEW LINES:
"""+newlines)

In [ ]:
AddToVirtual(virtual_dmc, "depmap-cn-data-9b9d", files=[('CCLE_gene_cn', 'dmc_'+release+'_gene_cn'),('CCLE_segment_cn', 'dmc_'+release+'_segs_cn')])

## We add to public as internal minus dmc embargoed and only cell lines from previous previous release (6 month)

In [ ]:
print(len(segmentcn))
segmentcn = segmentcn[segmentcn.DepMap_ID.isin(prevprev)]
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(set(wes_embargo))]
print(len(segmentcn))
segmentcn.to_csv('temp/public_' + release + '_segs_cn', index=False)
print(len(genecn))
genecn = genecn[genecn.index.isin(prevprev)]
genecn = genecn[~genecn.index.isin(set(wes_embargo))]
print(len(genecn))
genecn.to_csv('temp/public_'+release+'_gene_cn', index=True)

In [ ]:
prevcn = tc.get(name='depmap-wes-cn-data-97cc', version=31, file='public_'+prevname+"_segs_cn")
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.index))
print("new lines")
newlines = set(genecn.index) - set(prevcn.DepMap_ID) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname='depmap-wes-cn-data-97cc',
                    upload_file_path_dict={
                    'temp/public_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/public_'+release+'_segs_cn': 'TableCSV',
                   },
                  dataset_description="""
**** Versions 1-5 Public 18Q1****

Gene-level WES copy-number data for publicly accessible CCLE data. 

```

internal_lines <- readr::read_csv("~/Downloads/avana-broad-18q1_v2-sample-info.csv")$cell_line
public_lines <- readr::read_csv("~/Downloads/avana-public-tentative-18q1_v5-sample-info.csv")$cell_line
non_public_lines <- setdiff(internal_lines, public_lines)

full_cn_set <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', data.version=5, data.file='full_gene_CN')
source_info <- data.frame(source=gsub("_.*", "", row.names(full_cn_set)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(full_cn_set)),
                          row_idx=1:nrow(full_cn_set))
to_remove <- source_info %>%
  dplyr::filter(ccle_name %in% non_public_lines,
                source %in% c("ccleWES", "achillesWES"))
also_to_remove <- source_info %>%
                    dplyr::filter(source == "sangerWES")
indices_to_remove <- c(to_remove$row_idx, also_to_remove$row_idx) %>% unique()
indices_to_keep <- source_info %>%
  dplyr::filter(!(row_idx %in% indices_to_remove)) %>%
  dplyr::group_by(ccle_name) %>%
  dplyr::mutate(priority=ifelse(source == "snp", 4,
                                ifelse(source == "sangerWES", 3,
                                       ifelse(source == "ccleWES", 2, 1)))) %>%
  dplyr::filter(priority == min(priority)) %>%
  dplyr::ungroup()

public_cn <- full_cn_set[indices_to_keep$row_idx,]
source_info <- data.frame(source=gsub("_.*", "", row.names(public_cn)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(public_cn)))
public_cn %<>% magrittr::set_rownames(source_info$ccle_name)
```

CN data are on a log2 scale.

`WES_source_info` tracks the source data for each cell line. Sources are `snp`, `achillesWES`, `ccleWES`, and `sangerWES`

NOTE: Version 1 contained WES data from cell lines not available in the 18Q1 Public release. Versions 2-4 contained Sanger's WES CN data

**** Version 6 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=10, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

**** Version 7 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```


**** Version 8-9 Public 18Q3****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```
Version 8 the segment level CN for Sanger's data is off by a factor of 2, version 9 corrects this

includes cell lines that should not be public

**** Version 10 Public 18Q1, 18Q2, 18Q3****

__use version 10 for 18Q1, 18Q2 and 18Q3 datasets__ 

Version 10 is the most up-to-date version of "public\_18Q3\_gene\_cn.csv". The three datasets have been updated to remove cell lines that should not have been made public. They are named in the portal and google bucket for portal downloads as v2, e.g. public\_18Q3\_gene\_cn\_v2.csv.

__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 11 Public 18Q4****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 12-14 Public 19Q1****

version 14 contains the correct data for 19Q1

version 13 is the same as v12 except that it uses the original hg19 coordinates not hg38. The Achilles public data set uses the hg19 coordinates. 

**** Version 15-16 Public 19Q2****

__version 16 also adds the segment level copy number data__

**** Version 17-18 Public 19Q3****


**** Version 23 Public 19Q4****
adding new cell lines

**** Version 24 Internal 19Q4****
resolving problem with not having log2 transform 

**** Version 25 Internal 19Q4****
another issue in log transform

**** Version 26 Internal 19Q4****
unlog2 transforming segmentcn FINAL

**** Version 27 Internal 20Q1****
adding new samples

**** Version 28 Internal 20Q1****
log 2 transform issues

**** Version 29 Internal 20Q1****
readding one missing column in segments

**** Version 30 Internal 20Q2****
Adding new samples

**** Version 31 Internal 20Q2****
unknown changes

**** Version 32 Internal 20Q3****
same  as  20Q2 for this release. no new lines

**** Version 33 Internal 20Q3****
updated blacklist

**** Version 34 Internal 20Q3****
updated dmc list

## Gene level CN data:

__data is hg38 liftover__

__Description__: log2 + 1 gene level copy number data (data is log2 transformed with a __pseudocount of 1__ added). It uses hg19 coordinates. Also the segment level copy number data.

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean, CCLE\_name

NEW LINES:
"""+str(newlines))

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_public, "depmap-wes-cn-data-97cc", files=[('CCLE_gene_cn', 'public_'+release+'_gene_cn'),('CCLE_segment_cn', 'public_'+release+'_segs_cn')])